# Valence value regression based on Deap Dataset

## 0. This notebook is based on DEAP database

Anyone should refer to DEAP team first

@article{koelstra2012deap,
  title={Deap: A database for emotion analysis; using physiological signals},
  author={Koelstra, Sander and Muhl, Christian and Soleymani, Mohammad and Lee, Jong-Seok and Yazdani, Ashkan and Ebrahimi, Touradj and Pun, Thierry and Nijholt, Anton and Patras, Ioannis},
  journal={IEEE Transactions on Affective Computing},
  volume={3},
  number={1},
  pages={18--31},
  year={2012},
  publisher={IEEE}
}

## 1. Dependency
* numpy
* scipy
* mne (MNE-Python)
* scikit-learn

In [6]:
import numpy as np
import mne
# from scipy import signal
import pickle as pickle
import pandas as pd
import math

from sklearn import svm
from sklearn.preprocessing import normalize
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor

import os
#import tensorflow as tf
import time

## 2. Global Variables setup
File Name data\SXX.dat, XX \in [0,31]
* data: 40 x 40 x 8064: trial x channel x data
* label: 40 x 4: video/trial x label (valence, arousal, dominance, liking)

Channel Indice: {
* 1 : AF3; 2: F3; 3: F7; 4: FC5; 7: T7; 11: P7; 13: O1
* 17: AF4; 19: F4; 20: F8; 21: FC6; 25: T8; 29: P8; 31: O2 }

In [7]:
channel = [1,2,3,4,6,11,13,17,19,20,21,25,29,31] #14 Channels chosen to fit Emotiv Epoch+
band = [4,8,12,16,25,45] #5 bands
window_size = 256 #Averaging band power of 2 sec
step_size = 16 #Each 0.125 sec update once
sample_rate = 128 #Sampling rate of 128 Hz
subjectList = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32']
#List of subjects

## 3. FFT with MNE-Python
* [4-8]: theta band
* [8-12]: alpha band
* [12-16]: low beta band 
* [16-25]: high beta band
* [25-45]: gamma band

In [11]:
def bin_power(X, band, sample_rate):
    """
    计算频带功率，使用 MNE-Python 进行 EEG 信号处理
    替代 pyeeg.bin_power，使用 MNE 的功率谱密度计算方法
    
    Parameters:
    -----------
    X : array-like
        输入信号 (1D array)
    band : list
        频带边界，例如 [4, 8, 12, 16, 25, 45]
    sample_rate : int
        采样率 (Hz)
    
    Returns:
    --------
    power : array
        各频带的功率值
    frequencies : array
        频率数组（用于调试）
    """
    # 将数据转换为 MNE 格式 (需要是 2D: n_channels x n_times)
    X_2d = X.reshape(1, -1)  # 转换为 (1, n_samples)
    
    # 使用 MNE 计算功率谱密度
    # psd_array_welch 返回 (psd, freqs)，其中 psd 形状为 (n_channels, n_freqs)
    psd, frequencies = mne.time_frequency.psd_array_welch(
        X_2d, 
        sfreq=sample_rate,
        fmin=band[0],
        fmax=band[-1],
        n_fft=len(X),
        n_overlap=0,
        n_per_seg=len(X),
        verbose=False
    )
    
    # psd 形状是 (n_channels, n_freqs)，取第一个通道
    psd = psd[0]
    
    # 计算每个频带的功率
    power = []
    for i in range(len(band) - 1):
        # 找到频带范围内的频率索引
        freq_mask = (frequencies >= band[i]) & (frequencies < band[i + 1])
        if np.any(freq_mask):
            # 使用梯形积分计算频带功率
            band_power = np.trapezoid(psd[freq_mask], frequencies[freq_mask])
            power.append(band_power)
        else:
            # 如果没有找到频率点，返回 0
            power.append(0.0)
    
    return np.array(power), frequencies

def FFT_Processing (sub, channel, band, window_size, step_size, sample_rate):
    #data\s01.dat
    '''
    arguments:  string subject
                list channel indice
                list band
                int window size for FFT
                int step size for FFT
                int sample rate for FFT
    return:     void
    '''
    meta = []
    with open('data\s' + sub + '.dat', 'rb') as file:

        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1

        for i in range (0,40):
            # loop over 0-39 trails
            data = subject["data"][i]
            labels = subject["labels"][i]
            start = 0

            while start + window_size < data.shape[1]:
                meta_array = []
                meta_data = [] #meta vector for analysis
                for j in channel:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    Y, _ = bin_power(X, band, sample_rate) #FFT over 2 sec of channel j, in seq of theta, alpha, low beta, high beta, gamma
                    meta_data = meta_data + list(Y)

                meta_array.append(np.array(meta_data))
                meta_array.append(np.array(labels))  # 确保 labels 也是数组

                meta.append(meta_array)  # 保持为列表，不转换为数组
                start = start + step_size
                
        # 转换为数组时使用 object dtype 以支持不同形状的元素
        meta = np.array(meta, dtype=object)
        np.save('out\s' + sub, meta, allow_pickle=True, fix_imports=True)

def testing (M, L, model):
    '''
    arguments:  M: testing dataset
                L: testing dataset label
                model: scikit-learn model

    return:     void
    '''
    output = model.predict(M[0:78080:32])
    label = L[0:78080:32]

    k = 0
    l = 0

    for i in range(len(label)):
        k = k + (output[i] - label[i])*(output[i] - label[i]) #square difference 

        #a good guess
        if (output[i] > 5 and label[i] > 5):
            l = l + 1
        elif (output[i] < 5 and label[i] <5):
            l = l + 1

    print ("l2 error:", k/len(label), "classification accuracy:", l / len(label),l, len(label))

In [12]:
for subjects in subjectList:
    FFT_Processing (subjects, channel, band, window_size, step_size, sample_rate)

KeyboardInterrupt: 

## 3.Segment of preprocessed data
* training dataset: 75 %
* validation dataset: 12.5%
* testing dataset: 12.5%

Agrithom pool:
* Support Vector Machine (which kernal?)
* Ada-Boost

Best practice could be refered to this paper: 

@article{alarcao2017emotions,
  title={Emotions recognition using EEG signals: A survey},
  author={Alarcao, Soraia M and Fonseca, Manuel J},
  journal={IEEE Transactions on Affective Computing},
  year={2017},
  publisher={IEEE}
}

In [ ]:
#for subjects in subjectList:
data_training = []
label_training = []
data_testing = []
label_testing = []
data_validation = []
label_validation = []

for subjects in subjectList:

    with open('out\s' + subjects + '.npy', 'rb') as file:
        sub = np.load(file)
        for i in range (0,sub.shape[0]):
            if i % 8 == 0:
                data_testing.append(sub[i][0])
                label_testing.append(sub[i][1])
            elif i % 8 == 1:
                data_validation.append(sub[i][0])
                label_validation.append(sub[i][1])
            else:
                data_training.append(sub[i][0])
                label_training.append(sub[i][1])

np.save('out\data_training', np.array(data_training), allow_pickle=True, fix_imports=True)
np.save('out\label_training', np.array(label_training), allow_pickle=True, fix_imports=True)
print("training dataset:", np.array(data_training).shape, np.array(label_training).shape)

np.save('out\data_testing', np.array(data_testing), allow_pickle=True, fix_imports=True)
np.save('out\label_testing', np.array(label_testing), allow_pickle=True, fix_imports=True)
print("testing dataset:", np.array(data_testing).shape, np.array(label_testing).shape)

np.save('out\data_validation', np.array(data_validation), allow_pickle=True, fix_imports=True)
np.save('out\label_validation', np.array(label_validation), allow_pickle=True, fix_imports=True)
print("validation dataset:", np.array(data_validation).shape, np.array(label_validation).shape)

## 4.Regression
### 0. Loading Training and Testing dataset

In [ ]:
with open('out\data_training.npy', 'rb') as fileTrain:
    X  = np.load(fileTrain)
    
with open('out\label_training.npy', 'rb') as fileTrainL:
    Y  = np.load(fileTrainL)
    
X = normalize(X)
Z = np.ravel(Y[:, [1]])

Arousal_Train = np.ravel(Y[:, [0]])
Valence_Train = np.ravel(Y[:, [1]])
Domain_Train = np.ravel(Y[:, [2]])
Like_Train = np.ravel(Y[:, [3]])



with open('out\data_validation.npy', 'rb') as fileTrain:
    M  = np.load(fileTrain)
    
with open('out\label_validation.npy', 'rb') as fileTrainL:
    N  = np.load(fileTrainL)

M = normalize(M)
L = np.ravel(N[:, [1]])

Arousal_Test = np.ravel(N[:, [0]])
Valence_Test = np.ravel(N[:, [1]])
Domain_Test = np.ravel(N[:, [2]])
Like_Test = np.ravel(N[:, [3]])


### 1. Support Vector Regression
* default setting, l1 error: 1.621761042477756 classification error: 0.6057377049180328 1478 2440

In [ ]:
clf = svm.SVR()
clf.fit(X[0:468480:32], Z[0:468480:32])  

### 2. Random Forest Regression
* n_estimators = 10, sample rate = 1/32, l1 error: 1.137919672131145 classification accuracy: 0.7774590163934426 1897 2440
* n_estimators = 100, sample rate = 1/32, l1 error: 1.1029040163934432 classification accuracy: 0.8147540983606557 1988 2440
* n_estimators = 100, min_samples_leaf=10, sample rate = 1/32, l1 error: 1.274458098574928 classification accuracy: 0.7622950819672131 1860 2440
* n_estimators = 100, min_samples_leaf=50, sample rate = 1/32, l1 error: 1.4575897309409926 classification accuracy: 0.6823770491803278 1665 2440

* n_estimators = 250, sample rate = 1/32, l1 error: 1.0905590819672137 classification accuracy: 0.830327868852459 2026 2440
* n_estimators = 750, sample rate = 1/32, l1 error: 1.0953162021857932 classification accuracy: 0.8340163934426229 2035 2440
* n_estimators = 750, sample rate = 1/8, l1 error: l1 error: 1.066982950819674 classification accuracy: 0.8217213114754098 2005 2440
* __n_estimators = 512, sample rate = 1/32, l1 error: 1.092375304175206 classification accuracy: 0.8364754098360656 2041 2440
__



In [ ]:
Val_R = RandomForestRegressor(n_estimators=512, n_jobs=6)
Val_R.fit(X[0:468480:32], Valence_Train[0:468480:32])
testing (M, Valence_Test, Val_R)

In [ ]:
Aro_R = RandomForestRegressor(n_estimators=512, n_jobs=6)
Aro_R.fit(X[0:468480:32], Arousal_Train[0:468480:32])
testing (M, Arousal_Test, Aro_R)

In [ ]:
Dom_R = RandomForestRegressor(n_estimators=512, n_jobs=6)
Dom_R.fit(X[0:468480:32], Domain_Train[0:468480:32])
testing (M, Domain_Test, Dom_R)

In [ ]:
Lik_R = RandomForestRegressor(n_estimators=512, n_jobs=6)
Lik_R.fit(X[0:468480:32], Like_Train[0:468480:32])
testing (M, Like_Test, Lik_R)

### 3. AdaBoost Regression
* n = 50, lr = 1.0: l2 error: 3.8454054839726695 classification accuracy: 0.6147540983606558 1500 2440
* n = 50, lr = 1.0, square: l2 error: 4.015289218608164 classification accuracy: 0.5913934426229508 1443 2440
* n = 500, lr = 1.0: l2 error: 3.8861651269012594 classification accuracy: 0.6155737704918033 1502 2440
*
*

In [ ]:
clf = AdaBoostRegressor(n_estimators=5000, learning_rate=0.01)
clf.fit(X[0:468480:32], Z[0:468480:32])

### Calculating accuracy and loss

In [ ]:
output = Val_R.predict(M[0:78080:32])
label = L[0:78080:32]

k = 0
l = 0

for i in range(len(label)):
    k = k + (output[i] - label[i])*(output[i] - label[i]) #square difference 
    
    #a good guess
    if (output[i] > 5 and label[i] > 5):
        l = l + 1
    elif (output[i] < 5 and label[i] <5):
        l = l + 1

print ("l2 error:", k/len(label), "classification accuracy:", l / len(label),l, len(label))

### 4. ANN
* 500 epoch 0.005 128 - 256 - 256 - 128 loss = 3.1
* 3000 epoch 0.0001 256-512-512-256 Epoch: 3196 - Training Cost: 1.8372873067855835  Testing Cost: 2.231332540512085


In [ ]:
# Pull out columns for X (data to train with) and Y (value to predict)
X_training = X[0:468480:32]
Y_training = Z[0:468480:32]

# Pull out columns for X (data to train with) and Y (value to predict)
X_testing = M[0:78080:32]
Y_testing = L[0:78080:32]

# DO Scale both the training inputs and outputs
X_scaled_training = pd.DataFrame (data = X_training).values
Y_scaled_training = pd.DataFrame (data = Y_training).values

# It's very important that the training and test data are scaled with the same scaler.
X_scaled_testing = pd.DataFrame (data = X_testing).values
Y_scaled_testing = pd.DataFrame (data = Y_testing).values

In [ ]:
# Turn off TensorFlow warning messages in program output
from mne.utils.docs import tf


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Define model parameters
t = time.time()
learning_rate = 0.0001
training_epochs = 5000
display_step = 1

# Define how many inputs and outputs are in our neural network
number_of_inputs = 70
number_of_outputs = 1

# Define how many neurons we want in each layer of our neural network
layer_1_nodes = 512
layer_2_nodes = 1024
layer_3_nodes = 1024
layer_4_nodes = 512

# Section One: Define the layers of the neural network itself
RUN_NAME = str(int(round(t * 1000))) + '_' + str(layer_1_nodes) + '_' + str(layer_2_nodes) + '_' + str(layer_3_nodes) + '_' + str(layer_4_nodes) + '_' + str(learning_rate) + '_' + str(training_epochs) + '_' + 'Val'


# Input Layer
with tf.variable_scope('input'):
    X = tf.placeholder(tf.float32, shape=(None, number_of_inputs))

# Layer 1
with tf.variable_scope('layer_1'):
    weights = tf.get_variable("weights1", shape=[number_of_inputs, layer_1_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases1", shape=[layer_1_nodes], initializer=tf.zeros_initializer())
    layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)

# Layer 2
with tf.variable_scope('layer_2'):
    weights = tf.get_variable("weights2", shape=[layer_1_nodes, layer_2_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases2", shape=[layer_2_nodes], initializer=tf.zeros_initializer())
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)

# Layer 3
with tf.variable_scope('layer_3'):
    weights = tf.get_variable("weights3", shape=[layer_2_nodes, layer_3_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases3", shape=[layer_3_nodes], initializer=tf.zeros_initializer())
    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)

# Layer 4
with tf.variable_scope('layer_4'):
    weights = tf.get_variable("weights4", shape=[layer_3_nodes, layer_4_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases4", shape=[layer_4_nodes], initializer=tf.zeros_initializer())
    layer_4_output = tf.nn.relu(tf.matmul(layer_3_output, weights) + biases)

# Output Layer
with tf.variable_scope('output'):
    weights = tf.get_variable("weights5", shape=[layer_4_nodes, number_of_outputs], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases5", shape=[number_of_outputs], initializer=tf.zeros_initializer())
    prediction = tf.matmul(layer_4_output, weights) + biases

# Section Two: Define the cost function of the neural network that will be optimized during training

with tf.variable_scope('cost'):
    Y = tf.placeholder(tf.float32, shape=(None, 1))
    cost = tf.reduce_mean(tf.squared_difference(prediction, Y))

# Section Three: Define the optimizer function that will be run to optimize the neural network

with tf.variable_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

# Create a summary operation to log the progress of the network
with tf.variable_scope('logging'):
    tf.summary.scalar('current_cost', cost)
    summary = tf.summary.merge_all()

saver = tf.train.Saver()

# Initialize a session so that we can run TensorFlow operations
with tf.Session() as session:

    # Run the global variable initializer to initialize all variables and layers of the neural network
    session.run(tf.global_variables_initializer())

    # Create log file writers to record training progress.
    # We'll store training and testing log data separately.
    training_writer = tf.summary.FileWriter("./{}/logs/training".format(RUN_NAME), session.graph)
    testing_writer = tf.summary.FileWriter("./{}/logs/testing".format(RUN_NAME), session.graph)

    # Run the optimizer over and over to train the network.
    # One epoch is one full run through the training data set.
    for epoch in range(training_epochs):

        # Feed in the training data and do one step of neural network training
        session.run(optimizer, feed_dict={X: X_scaled_training, Y: Y_scaled_training})

        # Every few training steps, log our progress
        if epoch % display_step == 0:
            # Get the current accuracy scores by running the "cost" operation on the training and test data sets
            training_cost, training_summary = session.run([cost, summary], feed_dict={X: X_scaled_training, Y:Y_scaled_training})
            testing_cost, testing_summary = session.run([cost, summary], feed_dict={X: X_scaled_testing, Y:Y_scaled_testing})

            # Write the current training status to the log files (Which we can view with TensorBoard)
            training_writer.add_summary(training_summary, epoch)
            testing_writer.add_summary(testing_summary, epoch)

            # Print the current training status to the screen
            print("Epoch: {} - Training Cost: {}  Testing Cost: {}".format(epoch, training_cost, testing_cost))

    # Training is now complete!

    # Get the final accuracy scores by running the "cost" operation on the training and test data sets
    final_training_cost = session.run(cost, feed_dict={X: X_scaled_training, Y: Y_scaled_training})
    final_testing_cost = session.run(cost, feed_dict={X: X_scaled_testing, Y: Y_scaled_testing})

    print("Final Training cost: {}".format(final_training_cost))
    print("Final Testing cost: {}".format(final_testing_cost))

    save_path = saver.save(session, "./{}/logs/trained_model.ckpt".format(RUN_NAME))
    print("Model saved: {}".format(save_path))

    '''
    # Now that the neural network is trained, let's use it to make predictions for our test data.
    # Pass in the X testing data and run the "prediciton" operation
    Y_predicted_scaled = session.run(prediction, feed_dict={X: X_scaled_testing})
    # Unscale the data back to it's original units (dollars)
    Y_predicted = Y_scaler.inverse_transform(Y_predicted_scaled)
    real_earnings = test_data_df['total_earnings'].values[0]
    predicted_earnings = Y_predicted[0][0]
    print("The actual earnings of Game #1 were ${}".format(real_earnings))
    print("Our neural network predicted earnings of ${}".format(predicted_earnings))
    
'''
    model_builder = tf.saved_model.builder.SavedModelBuilder("./{}/exported_model".format(RUN_NAME))

    inputs = {
        'input': tf.saved_model.utils.build_tensor_info(X)
        }
    outputs = {
        'earnings': tf.saved_model.utils.build_tensor_info(prediction)
        }

    signature_def = tf.saved_model.signature_def_utils.build_signature_def(
        inputs=inputs,
        outputs=outputs,
        method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME
    )

    model_builder.add_meta_graph_and_variables(
        session,
        tags=[tf.saved_model.tag_constants.SERVING],
        signature_def_map={
            tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature_def
        }
    )

    model_builder.save()
    print('model saved')
